In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import os, time, sys, sklearn
from sklearn.externals import joblib
from rnn_functions import *


C:\Users\Workstation\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)



In [3]:

NROWS = 200000
TARGET_TO_PREDICT = "EURUSD"
FUTURE_PERIOD_PREDICT = 2

#Load
df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv", nrows = NROWS)
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")
print("Load: Done!")


Load: Done!


In [5]:
df

,AUDUSD,EURUSD,EURGBP,EURJPY,EWH,EWZ,FXI,IAU,EZU,KRE,...,VEA,VWO,XLE,XLI,XLV,XRT,XLY,VNQ,EWW,XBI
Date,,,,,,,,,,,,,,,,,,,,,
2008-05-05 09:45:00,-0.000318,0.000065,0.000000,0.000393,-0.002500,-0.000107,-0.000921,0.001168,0.000526,-0.001138,...,-0.000426,-0.000386,0.001551,0.001551,0.000626,-0.001809,-0.001523,-0.000299,0.000842,-0.002090
2008-05-05 09:50:00,0.000000,0.000258,0.000000,0.000000,0.000000,0.000427,0.001660,0.002334,0.001928,0.001425,...,0.001066,0.001545,0.000774,0.000774,0.000000,0.003625,0.002440,0.002090,0.002693,0.000000
2008-05-05 09:55:00,0.000000,-0.000278,-0.000025,0.000959,0.003008,0.002884,0.002025,-0.001164,-0.000350,0.002560,...,0.000000,0.000964,0.000258,0.000258,0.000939,0.000000,0.000000,0.000894,-0.000336,0.001047
2008-05-05 10:00:00,-0.000021,-0.000355,0.000025,-0.000307,-0.000500,0.001172,-0.000919,-0.001166,-0.000350,0.000568,...,0.000000,0.000385,-0.001031,-0.001031,-0.000313,0.000000,-0.000608,0.002084,-0.000168,0.000000
2008-05-05 10:05:00,0.000000,-0.001182,-0.000776,0.000000,-0.000500,0.000000,-0.000919,-0.001167,0.001400,-0.000567,...,-0.000639,-0.000770,-0.001033,-0.001033,0.000000,0.001204,0.000609,0.002080,0.001008,0.001569
2008-05-05 10:10:00,-0.000191,-0.000705,-0.000751,0.000430,0.003002,0.001915,0.002577,-0.003505,-0.001049,0.008229,...,0.001491,0.002697,0.003101,0.003101,0.002502,0.007216,0.005780,0.003261,0.003188,0.001567
2008-05-05 10:15:00,-0.001484,-0.000971,-0.000637,0.000000,-0.000499,-0.001487,-0.000918,0.002345,0.001400,-0.002815,...,-0.000213,-0.000769,-0.001030,-0.001030,0.000312,-0.001791,-0.000907,-0.000591,-0.001673,0.000000
2008-05-05 10:20:00,0.000531,0.000583,0.000255,-0.000651,-0.002495,-0.003084,-0.003124,-0.001170,-0.001573,-0.004516,...,-0.001064,-0.002692,-0.002063,-0.002063,-0.002183,-0.005383,-0.004239,-0.002513,-0.001173,-0.002086
2008-05-05 10:25:00,0.000743,0.000000,0.000637,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
#Import scaler
run_dir = "RUNS/EURUSD-60-2-2-10-0.3-0.0002-256-200-1538974780"
scaler = joblib.load(f'{run_dir}/rnn_scaler.pkl')
x_columns = joblib.load(f'{run_dir}/x_columns.pkl')
PARAMS_INFO = joblib.load(f'{run_dir}/PARAMS_INFO.pkl')
print("Import Scaler: Done!")

Import Scaler: Done!


In [10]:
PARAMS_INFO

{'BATCH_SIZE': 256,
 'DROPOUT': 0.3,
 'EPOCHS': 200,
 'FUTURE_PERIOD_PREDICT': 2,
 'LEARNING_RATE': 0.0002,
 'NAME': 'EURUSD-60-2-2-10-0.3-0.0002-256-200-1538974780',
 'NEURONS': 10,
 'NROWS': 200000,
 'NUMLAYER': 2,
 'PATIENCE': 30,
 'SEQ_LEN': 60,
 'TARGET_TO_PREDICT': 'EURUSD'}

In [11]:
df[x_columns] = scaler.transform(df[x_columns].values)
tdf = df[x_columns]
SEQ_LEN = PARAMS_INFO["SEQ_LEN"]

sequential_data = []
prev_days = deque(maxlen = SEQ_LEN)

sequential_list_dict = []
for i, j  in zip(tdf.values, tdf.index):   
    prev_days.append(i)
    if len(prev_days) == SEQ_LEN:
        sequential_list_dict.append(dict(t = j, x = np.array(prev_days)))


print("Transform: Done!")
print(sequential_list_dict[0])        



Transform: Done!
{'t': Timestamp('2008-05-05 14:40:00'), 'x': array([[-3.32807449e-01,  1.15665272e-01, -2.34639654e-03, ...,
        -1.30086591e-01,  4.72683339e-01, -1.65318473e+00],
       [ 4.59255657e-04,  4.60351274e-01, -2.34639654e-03, ...,
         9.19662687e-01,  1.50961667e+00, -3.04982860e-04],
       [ 4.59255657e-04, -4.93172929e-01, -5.30222955e-02, ...,
         3.93947338e-01, -1.87326653e-01,  8.27865655e-01],
       ...,
       [-4.86937578e-01, -1.37570235e+00, -1.59802856e+00, ...,
        -6.42020111e-02, -9.29467856e-02,  8.28733759e-01],
       [ 4.47886662e-02, -8.02798063e-01,  7.36993592e-02, ...,
        -1.29521225e-01,  5.63180982e-01, -3.04982860e-04],
       [ 4.59255657e-04,  3.45297708e-01, -2.34639654e-03, ...,
         6.64267208e-02, -2.80150019e-01, -3.04982860e-04]])}


In [12]:
model_input = "./tohost_model/RNN_Final-031-0.525.model"
rnn_model = tf.keras.models.load_model(filepath = model_input, custom_objects=None, compile=False)

print("Model Loaded!")

Model Loaded!


In [14]:
x_list_3d = []
t_list = []

for seq_dict in sequential_list_dict:
    t = seq_dict["t"]
    x = seq_dict["x"]
    t_list.append(t)
    x_list_3d.append(x)

x_list_3d = np.array(x_list_3d)

In [15]:
x_list_3d.shape

(199941, 60, 50)

In [16]:
y = rnn_model.predict(np.array(x_list_3d)) 

In [25]:
pred_df = pd.DataFrame(dict(Date = t_list, signal_raw = y.reshape((-1))))

In [26]:
instrument_name = PARAMS_INFO["TARGET_TO_PREDICT"]
pred_df.to_csv("rnn_signal_{}.csv".format(instrument_name), index = False)
